<a href="https://colab.research.google.com/github/alex-rsiqueira/finance-repo/blob/main/nubankpy%20to%20Bigquery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nubank Integration

## Setup

In [ ]:
!pip install -q google.appengine.api

In [ ]:
!pip install -q pynubank

In [ ]:
!pip install -q fsspec

In [ ]:
from pynubank.utils.certificate_generator import CertificateGenerator
from google.cloud.bigquery._helpers import _record_field_to_json
from pynubank import Nubank, MockHttpClient, NuException
#from google.appengine.api import app_identity
from colorama import init, Fore, Style
from google.cloud import bigquery
from google.colab import auth
from getpass import getpass
from io import StringIO
import pandas as pd
import numpy as np
import datetime
import random
import string
import json
import os

In [ ]:
carga_full = 1

## Authentication

### BigQuery

In [ ]:
auth.authenticate_user()
print('Authenticated')

Authenticated


### Nubank

#### Generate Certificate

In [ ]:
log(f'Starting {Fore.MAGENTA}{Style.DIM}PyNubank{Style.NORMAL}{Fore.LIGHTBLUE_EX} context creation.')

device_id = generate_random_id()

log(f'Generated random id: {device_id}')

cpf = input(f'[>] Enter your CPF(Numbers only): ')
password = getpass('[>] Enter your password (Used on the app/website): ')

generator = CertificateGenerator(cpf, password, device_id)

log('Requesting e-mail code')
try:
    email = generator.request_code()
except NuException:
    log(f'{Fore.RED}Failed to request code. Check your credentials!', Fore.RED)
    return

log(f'Email sent to {Fore.LIGHTBLACK_EX}{email}{Fore.LIGHTBLUE_EX}')
code = input('[>] Type the code received by email: ')

cert1, cert2 = generator.exchange_certs(code)

save_cert(cert1, 'cert.p12')

print(f'{Fore.GREEN}Certificates generated successfully. (cert.pem)')
print(f'{Fore.YELLOW}Warning, keep these certificates safe (Do not share or version in git)')

#### Use certificate to connect

In [ ]:
nu = Nubank()
refresh_token = nu.authenticate_with_cert('user', 'password', 'cert.p12')
#nu.authenticate_with_refresh_token(refresh_token, 'cert.p12')
print('Authenticated')

Authenticated


## Analysis

### Client

#### Retrieve client information

In [ ]:
client = nu.get_customer()

#### Delete unnecessary fields

In [ ]:
del client['_links']
del client['devices']
del client['primary_device']
del client['external_ids']
del client['channels']
#del client['documents']

# Convert dict to json
client_json = json.dumps(client,ensure_ascii=False)

#### Create dataframe - df_client

In [ ]:
df_client = pd.read_json(StringIO(client_json))
dtinsert = datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')
df_client['dtinsert'] = dtinsert
df_client = df_client.drop('documents',1)
display(df_client)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


,address_state,cpf,email,address_postcode,billing_address_line1,billing_address_state,address_number,address_ref_point,billing_address_city,phone,...,address_line1,profession,address_updated_at,gender,last_atualizacao_cadastral_at,billing_address_number,reported_income,invitations,address_city,dtinsert
0,SP,46983033892,alexsander_siqueira@outlook.com,4272300,Rua Vergueiro,SP,7170,estação Alto do Ipiranga,São Paulo,5511972854101,...,Rua Vergueiro,consultant,2021-03-29 13:42:24.976000+00:00,male,2020-07-20 14:01:10.703000+00:00,7170,7500,10,São Paulo,2022-03-15 03:13:39


In [ ]:
pd.set_option('display.max_rows', None)
display(df_client)

0    {'id': '5984b754-a5dc-47dc-ab74-b67206cb8618',...
Name: documents, dtype: object

#### Insert dataframe into BigQuery

In [ ]:
## Insere registros do arquivo no BigQuery
bq_client = bigquery.Client(project='finances-314506')
dataset = bq_client.dataset(dataset_id='raw').table('tb_nubank_client')
table = bq_client.get_table(dataset)
insert_row = bq_client.insert_rows_from_dataframe(table=table, dataframe=df_client)
print(f"Tabela populada com sucesso: tb_nubank_client")

Tabela populada com sucesso: tb_nubank_client


#### Save client ID for future tables

In [ ]:
client_id = df_client['id'][0]

### Credit Card

#### Retrieve credit card transactions

In [ ]:
transactions = nu.get_card_statements()

#Convert list to json
transactions_json = json.dumps(transactions)

#### Create dataframe - df_credit

In [ ]:
df_credit = pd.read_json(StringIO(transactions_json))

#### Delete unnecessary fields and replace NaN values

In [ ]:
del df_credit['_links']

#df_credit = df_credit.where(pd.notnull(df_credit), None)
df_credit = df_credit.astype(object).replace(np.nan, 'None')

display(df_credit)

,description,category,amount,time,source,title,amount_without_iof,account,details,id,tokenized,href
0,Ebanx *Sonyplaystatn,transaction,5118,2022-03-14T21:40:05Z,upfront_national,outros,5118.0,5984ced6-64d1-44ec-addb-932e839ba7cc,"{'status': 'unsettled', 'subcategory': 'card_n...",622fb636-cfea-4363-a9bd-082149014fdc,0.0,nuapp://transaction/622fb636-cfea-4363-a9bd-08...
1,Emporio Brasileiro,transaction,2440,2022-03-14T13:24:28Z,upfront_national,supermercado,2440.0,5984ced6-64d1-44ec-addb-932e839ba7cc,"{'status': 'unsettled', 'subcategory': 'card_p...",622f420c-4954-4db5-acfc-ce2802eaa78a,1.0,nuapp://transaction/622f420c-4954-4db5-acfc-ce...
2,Rappi*Mr Shu Bar Resta,transaction,4985,2022-03-13T00:48:10Z,upfront_national,restaurante,4985.0,5984ced6-64d1-44ec-addb-932e839ba7cc,"{'status': 'settled', 'subcategory': 'card_not...",622d3f4b-d0dd-425b-91df-56359767f2df,1.0,nuapp://transaction/622d3f4b-d0dd-425b-91df-56...
3,Uber *Uber *Trip,transaction,3495,2022-03-12T22:04:23Z,upfront_national,transporte,3495.0,5984ced6-64d1-44ec-addb-932e839ba7cc,"{'status': 'settled', 'subcategory': 'card_not...",622d18e8-ada7-421e-8c36-b27a0edc8eee,0.0,nuapp://transaction/622d18e8-ada7-421e-8c36-b2...
4,Uber *Uber *Trip,transaction,3603,2022-03-12T17:40:04Z,upfront_national,transporte,3603.0,5984ced6-64d1-44ec-addb-932e839ba7cc,"{'status': 'settled', 'subcategory': 'card_not...",622cdaf5-e4c1-4bbe-9ac5-cfbd6236147a,0.0,nuapp://transaction/622cdaf5-e4c1-4bbe-9ac5-cf...
...,...,...,...,...,...,...,...,...,...,...,...,...
3633,Viacao,transaction,3926,2017-08-19T21:16:02Z,None,transporte,None,None,"{'lat': -23.64610250575807, 'lon': -46.6423459...",5998aa92-e12c-4bf7-b5ac-eb1b9b4aa9c1,None,nuapp://transaction/5998aa92-e12c-4bf7-b5ac-eb...
3634,Thiago Alves Pereira D,transaction,1000,2017-08-18T16:14:20Z,None,restaurante,None,None,"{'lat': -23.53011105811048, 'lon': -46.6416518...",5997125e-13bf-4e62-bcf5-c6f3de231329,None,nuapp://transaction/5997125e-13bf-4e62-bcf5-c6...
3635,Trinity Cafe,transaction,2800,2017-08-18T16:07:25Z,None,restaurante,None,None,"{'lat': -23.52996586260791, 'lon': -46.6413158...",599710be-be68-4c92-beb6-adf7323bc899,None,nuapp://transaction/599710be-be68-4c92-beb6-ad...
3636,Doces do Porto,transaction,2230,2017-08-14T12:18:30Z,None,restaurante,None,None,{'subcategory': 'card_present'},59919516-f8a8-4bcf-a083-840ebb1306d5,None,nuapp://transaction/59919516-f8a8-4bcf-a083-84...


In [ ]:
df_credit['details']

0       {'status': 'unsettled', 'subcategory': 'card_n...
1       {'status': 'unsettled', 'subcategory': 'card_p...
2       {'status': 'settled', 'subcategory': 'card_not...
3       {'status': 'settled', 'subcategory': 'card_not...
4       {'status': 'settled', 'subcategory': 'card_not...
                              ...                        
3633    {'lat': -23.64610250575807, 'lon': -46.6423459...
3634    {'lat': -23.53011105811048, 'lon': -46.6416518...
3635    {'lat': -23.52996586260791, 'lon': -46.6413158...
3636                      {'subcategory': 'card_present'}
3637    {'lat': -23.5979747912135, 'lon': -46.67380336...
Name: details, Length: 3638, dtype: object

In [ ]:
dtinsert = datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')
df_credit['dtinsert'] = dtinsert
df_credit['id_client'] = str(client_id)

#### Insert dataframe into BigQuery

In [ ]:
df_credit

,description,category,amount,time,source,title,amount_without_iof,account,details,id,tokenized,href,dtinsert,id_client
0,Ebanx *Sonyplaystatn,transaction,5118,2022-03-14T21:40:05Z,upfront_national,outros,5118.0,5984ced6-64d1-44ec-addb-932e839ba7cc,"{'status': 'unsettled', 'subcategory': 'card_n...",622fb636-cfea-4363-a9bd-082149014fdc,0.0,nuapp://transaction/622fb636-cfea-4363-a9bd-08...,2022-03-15 03:17:53,5984b6bd-b065-460a-a9aa-bb4ec3af99de
1,Emporio Brasileiro,transaction,2440,2022-03-14T13:24:28Z,upfront_national,supermercado,2440.0,5984ced6-64d1-44ec-addb-932e839ba7cc,"{'status': 'unsettled', 'subcategory': 'card_p...",622f420c-4954-4db5-acfc-ce2802eaa78a,1.0,nuapp://transaction/622f420c-4954-4db5-acfc-ce...,2022-03-15 03:17:53,5984b6bd-b065-460a-a9aa-bb4ec3af99de
2,Rappi*Mr Shu Bar Resta,transaction,4985,2022-03-13T00:48:10Z,upfront_national,restaurante,4985.0,5984ced6-64d1-44ec-addb-932e839ba7cc,"{'status': 'settled', 'subcategory': 'card_not...",622d3f4b-d0dd-425b-91df-56359767f2df,1.0,nuapp://transaction/622d3f4b-d0dd-425b-91df-56...,2022-03-15 03:17:53,5984b6bd-b065-460a-a9aa-bb4ec3af99de
3,Uber *Uber *Trip,transaction,3495,2022-03-12T22:04:23Z,upfront_national,transporte,3495.0,5984ced6-64d1-44ec-addb-932e839ba7cc,"{'status': 'settled', 'subcategory': 'card_not...",622d18e8-ada7-421e-8c36-b27a0edc8eee,0.0,nuapp://transaction/622d18e8-ada7-421e-8c36-b2...,2022-03-15 03:17:53,5984b6bd-b065-460a-a9aa-bb4ec3af99de
4,Uber *Uber *Trip,transaction,3603,2022-03-12T17:40:04Z,upfront_national,transporte,3603.0,5984ced6-64d1-44ec-addb-932e839ba7cc,"{'status': 'settled', 'subcategory': 'card_not...",622cdaf5-e4c1-4bbe-9ac5-cfbd6236147a,0.0,nuapp://transaction/622cdaf5-e4c1-4bbe-9ac5-cf...,2022-03-15 03:17:53,5984b6bd-b065-460a-a9aa-bb4ec3af99de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3633,Viacao,transaction,3926,2017-08-19T21:16:02Z,None,transporte,None,None,"{'lat': -23.64610250575807, 'lon': -46.6423459...",5998aa92-e12c-4bf7-b5ac-eb1b9b4aa9c1,None,nuapp://transaction/5998aa92-e12c-4bf7-b5ac-eb...,2022-03-15 03:17:53,5984b6bd-b065-460a-a9aa-bb4ec3af99de
3634,Thiago Alves Pereira D,transaction,1000,2017-08-18T16:14:20Z,None,restaurante,None,None,"{'lat': -23.53011105811048, 'lon': -46.6416518...",5997125e-13bf-4e62-bcf5-c6f3de231329,None,nuapp://transaction/5997125e-13bf-4e62-bcf5-c6...,2022-03-15 03:17:53,5984b6bd-b065-460a-a9aa-bb4ec3af99de
3635,Trinity Cafe,transaction,2800,2017-08-18T16:07:25Z,None,restaurante,None,None,"{'lat': -23.52996586260791, 'lon': -46.6413158...",599710be-be68-4c92-beb6-adf7323bc899,None,nuapp://transaction/599710be-be68-4c92-beb6-ad...,2022-03-15 03:17:53,5984b6bd-b065-460a-a9aa-bb4ec3af99de
3636,Doces do Porto,transaction,2230,2017-08-14T12:18:30Z,None,restaurante,None,None,{'subcategory': 'card_present'},59919516-f8a8-4bcf-a083-840ebb1306d5,None,nuapp://transaction/59919516-f8a8-4bcf-a083-84...,2022-03-15 03:17:53,5984b6bd-b065-460a-a9aa-bb4ec3af99de


In [ ]:
## Insere registros do arquivo no BigQuery
bq_client = bigquery.Client(project='finances-314506')
dataset = bq_client.dataset(dataset_id='raw').table('tb_nubank_credit')
table = bq_client.get_table(dataset)
insert_row = bq_client.insert_rows_from_dataframe(table=table, dataframe=df_credit)
print(f"Tabela populada com sucesso: tb_nubank_credit")

Tabela populada com sucesso: tb_nubank_credit


In [ ]:
# Agrupa pelo campo "title" que é a categoria e soma os valores
df = pd.read_json(StringIO(transactions_json)).groupby(['title']).sum()

# Plota o gráfico baseado no campo amount
df['amount'].plot.pie(figsize=(6, 6), autopct='%.2f')

#### Retrieve information from specific bill

In [ ]:
# Lista de dicionários contendo todas as faturas do seu cartão de crédito
bills = nu.get_bills()

# Retorna um dicionário contendo os detalhes de uma fatura retornada por get_bills()
#bill_details = nu.get_bill_details(bills[2])

In [ ]:
bills[4]['summary']

{'close_date': '2022-05-09',
 'due_date': '2022-05-16',
 'effective_due_date': '2022-05-16',
 'interest': 0,
 'interest_rate': '0.14',
 'minimum_payment': 0,
 'open_date': '2022-04-09',
 'paid': 0,
 'past_balance': 0,
 'total_balance': 84820,
 'total_cumulative': 84820}

In [ ]:
df_bills = pd.DataFrame(bills)
df_bills

,state,summary,_links,id,installment_button
0,future,"{'due_date': '2022-09-16', 'close_date': '2022...",{},NaN,NaN
1,future,"{'due_date': '2022-08-16', 'close_date': '2022...",{},NaN,NaN
2,future,"{'due_date': '2022-07-16', 'close_date': '2022...",{},NaN,NaN
3,future,"{'due_date': '2022-06-16', 'close_date': '2022...",{},NaN,NaN
4,future,"{'due_date': '2022-05-16', 'close_date': '2022...",{},NaN,NaN
...,...,...,...,...,...
56,overdue,"{'due_date': '2018-01-19', 'close_date': '2018...",{'self': {'href': 'https://prod-s0-facade.nuba...,5a582dc5-82a7-4ed0-8555-c17b0e53cb22,NaN
57,overdue,"{'due_date': '2017-12-19', 'close_date': '2017...",{'self': {'href': 'https://prod-s0-facade.nuba...,5a2f4f50-c2f2-479d-9da5-521dedde528b,NaN
58,overdue,"{'due_date': '2017-11-19', 'close_date': '2017...",{'self': {'href': 'https://prod-s0-facade.nuba...,5a07bf26-45a5-4e2d-9201-360f1c57f7bc,NaN
59,overdue,"{'due_date': '2017-10-19', 'close_date': '2017...",{'self': {'href': 'https://prod-s0-facade.nuba...,59dee360-f9a9-4772-96d5-a53e627f6be6,NaN


In [ ]:
df_bills = pd.DataFrame(bills[6]['summary'], index=[0])

In [ ]:
df_bills = df_bills[['open_date','due_date','total_balance','minimum_payment']]

In [ ]:
df_bills

,open_date,due_date,total_balance,minimum_payment
0,2022-02-09,2022-03-16,887792,133169


In [ ]:
int(datetime.datetime.today().strftime('%Y%m'))

202203

In [ ]:
df_bills.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 0 to 0
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   open_date        1 non-null      object
 1   due_date         1 non-null      object
 2   total_balance    1 non-null      int64 
 3   minimum_payment  1 non-null      int64 
 4   bill_id          1 non-null      object
 5   bill_competence  1 non-null      object
 6   state            1 non-null      object
 7   id_client        1 non-null      object
 8   dtinsert         1 non-null      object
dtypes: int64(2), object(7)
memory usage: 80.0+ bytes


In [ ]:
for i in range(0,len(bills)):
    bill_state = bills[i]['state']
    if bill_state != 'future':
        #print(bill_state)
        if bill_state in ('closed','overdue'):
            bill_href = bills[i]['_links']['self']['href']
            bill_id = bill_href[::-1][:bill_href[::-1].find('/')][::-1]
        elif bill_state == 'open':
            bill_href = bills[i]['_links']['self']['href']
            bill_id = bill_href[::-1][bill_href[::-1].find('sllib/')+6:bill_href[::-1].find('/stnuocca')][::-1]
        else:
            bill_id = None

        bill_competence = bills[i]['summary']['open_date'][:-3]

        if (int(bill_competence.replace('-','')) > int(datetime.datetime.today().strftime('%Y%m'))-2 or carga_full == 1):
            df_bills = pd.DataFrame(bills[i]['summary'], index=[0])
            df_bills = df_bills[['open_date','due_date','total_balance','minimum_payment']]
            df_bills['bill_id'] = bill_id
            df_bills['bill_competence'] = bill_competence
            df_bills['state'] = bill_state
            df_bills['id_client'] = client_id
            dtinsert = datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')
            df_bills['dtinsert'] = dtinsert
            
            ## Insere registros no BigQuery
            bq_client = bigquery.Client(project='finances-314506')
            dataset = bq_client.dataset(dataset_id='raw').table('tb_nubank_bills')
            table = bq_client.get_table(dataset)
            insert_row = bq_client.insert_rows_from_dataframe(table=table, dataframe=df_bills)
            print(f"Tabela populada com sucesso: tb_nubank_bills")


            bill_details = nu.get_bill_details(bills[i]) 

            # Cria dataframe com lançamentos de cada fatura
            df_bill_statements = pd.DataFrame(bill_details['bill']['line_items'])
            df_bill_statements['bill_id'] = bill_id
            df_bill_statements['bill_competence'] = bill_competence
            
            dtinsert = datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')
            df_bill_statements['dtinsert'] = dtinsert
            df_bill_statements['id_client'] = client_id
            df_bill_statements = df_bill_statements.where(pd.notnull(df_bill_statements), None)
            df_bill_statements = df_bill_statements.astype(object).replace(np.nan, 'None')

            ## Insere registros no BigQuery
            bq_client = bigquery.Client(project='finances-314506')
            dataset = bq_client.dataset(dataset_id='raw').table('tb_nubank_bill_statements')
            table = bq_client.get_table(dataset)
            insert_row = bq_client.insert_rows_from_dataframe(table=table, dataframe=df_bill_statements)
            print(f"Tabela populada com sucesso: tb_nubank_bill_statements")
    
    if bill_state == 'future':
        bill_id = None
        bill_competence = bills[i]['summary']['open_date'][:-3]

        df_bills = pd.DataFrame(bills[i]['summary'], index=[0])
        df_bills = df_bills[['open_date','due_date','total_balance','minimum_payment']]
        df_bills['bill_id'] = bill_id
        df_bills['bill_competence'] = bill_competence
        df_bills['state'] = bill_state
        dtinsert = datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')
        df_bills['dtinsert'] = dtinsert
        df_bills['id_client'] = client_id
        
        ## Insere registros no BigQuery
        bq_client = bigquery.Client(project='finances-314506')
        dataset = bq_client.dataset(dataset_id='raw').table('tb_nubank_bills')
        table = bq_client.get_table(dataset)
        insert_row = bq_client.insert_rows_from_dataframe(table=table, dataframe=df_bills)
        print(f"Tabela populada com sucesso: tb_nubank_bills")

Tabela populada com sucesso: tb_nubank_bills
Tabela populada com sucesso: tb_nubank_bills
Tabela populada com sucesso: tb_nubank_bills
Tabela populada com sucesso: tb_nubank_bills
Tabela populada com sucesso: tb_nubank_bills
Tabela populada com sucesso: tb_nubank_bills
Tabela populada com sucesso: tb_nubank_bill_statements
Tabela populada com sucesso: tb_nubank_bills
Tabela populada com sucesso: tb_nubank_bill_statements
Tabela populada com sucesso: tb_nubank_bills
Tabela populada com sucesso: tb_nubank_bill_statements
Tabela populada com sucesso: tb_nubank_bills
Tabela populada com sucesso: tb_nubank_bill_statements
Tabela populada com sucesso: tb_nubank_bills
Tabela populada com sucesso: tb_nubank_bill_statements
Tabela populada com sucesso: tb_nubank_bills
Tabela populada com sucesso: tb_nubank_bill_statements
Tabela populada com sucesso: tb_nubank_bills
Tabela populada com sucesso: tb_nubank_bill_statements
Tabela populada com sucesso: tb_nubank_bills
Tabela populada com sucesso: t





#### Check balance from specific bill

In [ ]:
print(sum([t['amount'] for t in bill_details['bill']['line_items']])/100)

558.53


### Account

#### Check account balance

In [ ]:
print(nu.get_account_balance())

2.43


#### Retrieve account transactions

In [ ]:
account_statement = nu.get_account_feed()

account_statement_json = json.dumps(account_statement)

#### Create dataframe - df_account_statement

In [ ]:
df_account_statement = pd.read_json(StringIO(account_statement_json))
#display(df_account_statement)

#### Treat dataframe

In [ ]:
df_account_statement

,id,__typename,title,detail,postDate,amount,originAccount,destinationAccount
0,622ffa28-cefa-4b92-b6a5-7421185a2f52,LendingTransferOutEvent,Parcela atrasada,1ª parcela,2022-03-14,None,None,None
1,622cd4d2-65df-4863-abce-26563e6f0766,GenericFeedEvent,Transferência enviada,"Higor Teixeira da Silva\nR$ 70,00",2022-03-12,None,None,None
2,622b36f1-cd1f-40eb-8e66-4bdfd7c3b85b,GenericFeedEvent,Transferência enviada,"TELEFONICA BRASIL S A \nR$ 106,90",2022-03-11,None,None,None
3,622a0530-ad7c-4b2d-938c-4e2319a923c4,GenericFeedEvent,Transferência enviada,"Larissa Passine de Souza\nR$ 57,00",2022-03-10,None,None,None
4,6229f936-77e4-40b4-854d-8899115ab4c9,GenericFeedEvent,Transferência enviada,"ROBERTA SANTOS OGATA\nR$ 600,00",2022-03-10,None,None,None
...,...,...,...,...,...,...,...,...
1122,5c584a60-58f6-43bd-8cb1-2ebea16e503f,TransferInEvent,Transferência recebida,"R$ 2.362,00",2019-02-04,2362.0,{'name': 'Alexsander Rodrigues de Siqueira'},None
1123,5c584ae6-bc9f-422c-8a70-046963168888,BillPaymentEvent,Pagamento da fatura,"Cartão Nubank - R$ 2.361,16",2019-02-04,2361.16,None,None
1124,5c0950c3-7fe4-49e4-9239-f9ccc0389fc5,BillPaymentEvent,Pagamento da fatura,"Cartão Nubank - R$ 1.284,46",2018-12-06,1284.46,None,None
1125,5c09504e-1b25-4c40-9fd3-d69a4811b54d,TransferInEvent,Transferência recebida,"R$ 1.285,00",2018-12-06,1285.0,{'name': 'Alexsander Rodrigues de Siqueira'},None


In [ ]:
# replace NaN values
df_account_statement = df_account_statement.where(pd.notnull(df_account_statement), None)
df_account_statement['amount'] = df_account_statement['amount'].astype(object).replace(np.nan, 'None')

In [ ]:
# rename column
df_account_statement = df_account_statement.rename(columns={'__typename': 'typename'})

# retrieve the name from the dict value
df_account_statement['originAccount'] = df_account_statement['originAccount'].apply(lambda x: x.get('name') if x != None else x)
df_account_statement['destinationAccount'] = df_account_statement['destinationAccount'].apply(lambda x: x.get('name') if x != None else x)

dtinsert = datetime.datetime.today().strftime('%Y-%m-%d %H:%M:%S')
df_account_statement['dtinsert'] = dtinsert
df_account_statement['id_client'] = client_id

display(df_account_statement)

,id,typename,title,detail,postDate,amount,originAccount,destinationAccount,dtinsert,id_client
0,622ffa28-cefa-4b92-b6a5-7421185a2f52,LendingTransferOutEvent,Parcela atrasada,1ª parcela,2022-03-14,None,None,None,2022-03-15 03:38:42,5984b6bd-b065-460a-a9aa-bb4ec3af99de
1,622cd4d2-65df-4863-abce-26563e6f0766,GenericFeedEvent,Transferência enviada,"Higor Teixeira da Silva\nR$ 70,00",2022-03-12,None,None,None,2022-03-15 03:38:42,5984b6bd-b065-460a-a9aa-bb4ec3af99de
2,622b36f1-cd1f-40eb-8e66-4bdfd7c3b85b,GenericFeedEvent,Transferência enviada,"TELEFONICA BRASIL S A \nR$ 106,90",2022-03-11,None,None,None,2022-03-15 03:38:42,5984b6bd-b065-460a-a9aa-bb4ec3af99de
3,622a0530-ad7c-4b2d-938c-4e2319a923c4,GenericFeedEvent,Transferência enviada,"Larissa Passine de Souza\nR$ 57,00",2022-03-10,None,None,None,2022-03-15 03:38:42,5984b6bd-b065-460a-a9aa-bb4ec3af99de
4,6229f936-77e4-40b4-854d-8899115ab4c9,GenericFeedEvent,Transferência enviada,"ROBERTA SANTOS OGATA\nR$ 600,00",2022-03-10,None,None,None,2022-03-15 03:38:42,5984b6bd-b065-460a-a9aa-bb4ec3af99de
...,...,...,...,...,...,...,...,...,...,...
1122,5c584a60-58f6-43bd-8cb1-2ebea16e503f,TransferInEvent,Transferência recebida,"R$ 2.362,00",2019-02-04,2362.0,Alexsander Rodrigues de Siqueira,None,2022-03-15 03:38:42,5984b6bd-b065-460a-a9aa-bb4ec3af99de
1123,5c584ae6-bc9f-422c-8a70-046963168888,BillPaymentEvent,Pagamento da fatura,"Cartão Nubank - R$ 2.361,16",2019-02-04,2361.16,None,None,2022-03-15 03:38:42,5984b6bd-b065-460a-a9aa-bb4ec3af99de
1124,5c0950c3-7fe4-49e4-9239-f9ccc0389fc5,BillPaymentEvent,Pagamento da fatura,"Cartão Nubank - R$ 1.284,46",2018-12-06,1284.46,None,None,2022-03-15 03:38:42,5984b6bd-b065-460a-a9aa-bb4ec3af99de
1125,5c09504e-1b25-4c40-9fd3-d69a4811b54d,TransferInEvent,Transferência recebida,"R$ 1.285,00",2018-12-06,1285.0,Alexsander Rodrigues de Siqueira,None,2022-03-15 03:38:42,5984b6bd-b065-460a-a9aa-bb4ec3af99de


#### Insert dataframe into BigQuery

In [ ]:
## Insere registros do arquivo no BigQuery
bq_client = bigquery.Client(project='finances-314506')
dataset = bq_client.dataset(dataset_id='raw').table('tb_nubank_account')
table = bq_client.get_table(dataset)
insert_row = bq_client.insert_rows_from_dataframe(table=table, dataframe=df_account_statement)
print(f"Tabela populada com sucesso: tb_nubank_account")

Tabela populada com sucesso: tb_nubank_account
